In [ ]:
# notebook settings
%load_ext autoreload
%autoreload 2

# external imports
import numpy as np
import sympy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
from copy import copy
from math import floor

# internal imports
from pympc.geometry.polyhedron import Polyhedron
from pympc.dynamics.discrete_time_systems import LinearSystem, AffineSystem, PieceWiseAffineSystem
from pympc.control.hybrid_benchmark.controllers import HybridModelPredictiveController
from pympc.plot import plot_input_sequence, plot_state_trajectory, plot_output_trajectory
from pympc.control.hybrid_benchmark.utils import get_constraint_set, remove_redundant_inequalities_fast, convex_hull_method_fast

# Problem set-up

In [ ]:
# numeric parameters of the system
m = 1.
r = .1
I = .4*m*r**2.
d = .5
mu = .2
g = 10.
h = .05

In [ ]:
# symbolic state
xb, yb, tb, xp, yp, xdb, ydb, tdb, xdp, ydp = sp.symbols('xb yb tb xp yp xdb ydb tdb xdp ydp')
state = sp.Matrix([xb, yb, tb, xp, yp, xdb, ydb, tdb, xdp, ydp])

# symbolic input
xd2p, yd2p = sp.symbols('xd2p yd2p')
input = sp.Matrix([xd2p, yd2p])

# contact forces
ft, fn = sp.symbols('ft fn')

# equilibrium point
origin = {s:0 for s in state}
origin.update({i:0 for i in input})

In [ ]:
# ball velocity update
xdb_next = xdb + h*ft/m
ydb_next = ydb + h*fn/m - h*g
tdb_next = tdb + r*h*ft/I

# paddle velocity update
xdp_next = xdp + h*xd2p
ydp_next = ydp + h*yd2p

# ball position update
xb_next = xb + h*xdb_next
yb_next = yb + h*ydb_next
tb_next = tb + h*tdb_next

# paddle position update
xp_next = xp + h*xdp_next
yp_next = yp + h*ydp_next

# state update
state_next = sp.Matrix([xb_next, yb_next, tb_next, xp_next, yp_next, xdb_next, ydb_next, tdb_next, xdp_next, ydp_next])

In [ ]:
# relative tangential velocity (ball wrt paddle)
rel_vel = xdb_next + r*tdb_next - xdp_next

# gap function
gap = yb_next - yp_next
gap_no_force = gap.subs({ft: 0., fn: 0.})

In [ ]:
# extracts affine system from the next state value
def get_affine_system(forces):
    x = state_next.subs(forces)
    A = np.array(x.jacobian(state)).astype(np.float64)
    B = np.array(x.jacobian(input)).astype(np.float64)
    c = np.array(x.subs(origin)).astype(np.float64).flatten()
    return AffineSystem(A, B, c)

# returns A and -b for the expression A*(state,input)+b
def get_matrices(lin_expr):
    A = np.array(sp.Matrix([lin_expr]).jacobian(state.col_join(input))).astype(np.float64)
    b = np.array([lin_expr.subs(origin)]).astype(np.float64)
    return A, -b

In [ ]:
# state bounds
x_max = np.array([1., 1., 3.*np.pi, 1., 1., 5., 5., 20., 5., 5.])
x_min = - x_max

# input bounds
u_max = np.array([5., 5.])
u_min = - u_max

# domain bounds
xu_min = np.concatenate((x_min, u_min))
xu_max = np.concatenate((x_max, u_max))

In [ ]:
# discrete time dynamics in mode 1
# (ball in the air)

# set forces to zero
forces = {ft: 0., fn: 0.}

# get dynamics
S1 = get_affine_system(forces)

# build domain
D1 = Polyhedron.from_bounds(xu_min, xu_max)

# - gap <= 0
D1.add_inequality(*get_matrices(-gap_no_force))

# check domain
assert D1.bounded
assert not D1.empty

In [ ]:
# discrete time dynamics in mode 2
# (ball in contact with sticking)

# solve for the contact forces
ft_val = sp.solve(sp.Eq(rel_vel, 0), ft)[0]
fn_val = sp.solve(sp.Eq(gap, 0), fn)[0]
forces = {ft: ft_val, fn: fn_val}

# get dynamics
S2 = get_affine_system(forces)

# build domain
D2 = Polyhedron.from_bounds(xu_min, xu_max)

# gap <= 0
D2.add_inequality(*get_matrices(gap_no_force))

# ball not falling down the paddle
D2.add_inequality(*get_matrices(xb - xp - d))
D2.add_inequality(*get_matrices(xp - xb - d))

# friction cone
D2.add_inequality(*get_matrices(ft_val - mu*fn_val))
D2.add_inequality(*get_matrices(- ft_val - mu*fn_val))

# check domain
assert D2.bounded
assert not D2.empty

In [ ]:
# discrete time dynamics in mode 3
# (ball in contact and sliding right wrt paddle)

# boundary of the friction cone
forces = {ft: -mu*fn_val, fn: fn_val}

# get dynamics
S3 = get_affine_system(forces)

# build domain
D3 = Polyhedron.from_bounds(xu_min, xu_max)

# gap <= 0
D3.add_inequality(*get_matrices(gap_no_force))

# ball not falling down the paddle
D2.add_inequality(*get_matrices(xb - xp - d))
D2.add_inequality(*get_matrices(xp - xb - d))

# positive relative velocity
D3.add_inequality(*get_matrices(-rel_vel.subs(ft, -mu*fn_val)))

# check domain
assert D3.bounded
assert not D3.empty

In [ ]:
# discrete time dynamics in mode 4
# (ball in contact and sliding left wrt paddle)

# boundary of the friction cone
forces = {ft: mu*fn_val, fn: fn_val}

# get dynamics
S4 = get_affine_system(forces)

# build domain
D4 = Polyhedron.from_bounds(xu_min, xu_max)

# gap <= 0
D4.add_inequality(*get_matrices(gap_no_force))

# ball not falling down the paddle
D2.add_inequality(*get_matrices(xb - xp - d))
D2.add_inequality(*get_matrices(xp - xb - d))

# negative relative velocity
D4.add_inequality(*get_matrices(rel_vel.subs(ft, mu*fn_val)))

# check domain
assert D4.bounded
assert not D4.empty

In [ ]:
# list of dynamics
S_list = [S1, S2, S3, S4]

# list of domains
D_list = [D1, D2, D3, D4]

# PWA system
S = PieceWiseAffineSystem(S_list, D_list)

In [ ]:
# PWA system
S = PieceWiseAffineSystem(S_list, D_list)

# controller parameters
N = 100
Q = np.diag([1.,1.,1.,1.,1.,1.,1.,1.,1.,1.])*h
R = np.diag([1.,1.])*h

# terminal set and cost
P = Q
xN_min = np.array([-d,0.,0.,0.,0.,0.,0.,0.,0.,0.])
xN_max = np.array([d,0.,0.,0.,0.,0.,0.,0.,0.,0.])
X_N = Polyhedron.from_bounds(xN_min, xN_max)

In [ ]:
controller = HybridModelPredictiveController(S, N, Q, R, P, X_N, method='Convex hull')
# controller.prog.setParam('Heuristics', 0)

In [ ]:
# initial condition
x0 = np.array([0.,0.,0.,0.,0.,0.,0.,0.,0.,0.])
x0 = np.array([0.,0.,np.pi,0.,0.,0.,0.,0.,0.,0.])

In [ ]:
u, x, ms, cost = controller.feedforward(x0)

In [ ]:
plot_input_sequence(u, h, (u_min, u_max))

In [ ]:
plot_state_trajectory(x, h, (x_min, x_max))

In [ ]:
# x0 = np.array([0.,0.,0.,0.,0.,0.,0.,7.2,0.,0.])
# u = [np.array([0.,0.])]*50
x, ms = S.simulate(x0, u)
print ms

# Animation

In [ ]:
import meshcat
from meshcat.geometry import Box, Sphere, Cylinder, MeshLambertMaterial
from meshcat.animation import Animation
import meshcat.transformations as tf

In [ ]:
vis = meshcat.Visualizer()
# vis.jupyter_cell()
vis.open()

In [ ]:
paddle_z = .05
paddle_x = .3
red = 0xff2222
blue = 0x2222ff
green = 0x22ff22
vis['paddle'].set_object(Box([paddle_x, d*2., paddle_z]), MeshLambertMaterial(color=red))
vis['ball'].set_object(Sphere(r), MeshLambertMaterial(color=blue))
vis['ball_orientation'].set_object(Cylinder(.01, r), MeshLambertMaterial(color=green))

In [ ]:
anim = Animation()
for t, xt in enumerate(x):
    with anim.at_frame(vis, t*h*30) as frame:
        frame['paddle'].set_transform(
            tf.translation_matrix([0, xt[3], xt[4]-paddle_z/2.])
        )
        frame['ball'].set_transform(
            tf.translation_matrix([0, xt[0], xt[1]+r])
        )
        frame['ball_orientation'].set_transform(
            tf.translation_matrix([0, xt[0], xt[1]+r]).dot(
                tf.rotation_matrix(xt[2], [1.,0.,0.])
            )
        )
vis.set_animation(anim)